In [24]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chisquare

In [2]:
from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [3]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [4]:
def gaussian_reader(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df

In [5]:
df = csv_format_discrete('alarm_10000.dat')
dataframe = df
df

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


In [37]:
def chi_square_CI(df):
#the null hypothesis is that they are independent of each other
#if true, accept null hypothesis
#if false, reject null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    empty_5 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
    for x,y,i in zip(empty_1, empty_2, empty):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.95, boolean=False) #returns chi, p_value, dof
        empty_3.append([i , chisquare])
    for value in range(len(empty_3)):
        empty_4.append(value)
    for x,y in zip(empty_4, empty_3):
        empty_5.append([x,y])
    return empty_5
#if the chi_squared test gives true, then p_value is > significance level, hence we accept the null hypothesis, false otherwise

In [38]:
chi_square_CI(df)

[[0, [('asia', 'tub'), (8.017532239981868, 0.0046326652461664624, 1)]],
 [1, [('asia', 'smoke'), (1.057205091929743, 0.3038535053191063, 1)]],
 [2, [('asia', 'lung'), (0.0, 1.0, 1)]],
 [3, [('asia', 'bronc'), (0.0, 1.0, 1)]],
 [4, [('asia', 'either'), (1.8288335974484768, 0.17626537727626068, 1)]],
 [5, [('asia', 'xray'), (0.0, 1.0, 1)]],
 [6, [('asia', 'dysp'), (0.27203477171555374, 0.6019700848684595, 1)]],
 [7, [('tub', 'smoke'), (2.302538990327108, 0.12916271166867047, 1)]],
 [8, [('tub', 'lung'), (0.829989000562263, 0.36227570962559885, 1)]],
 [9, [('tub', 'bronc'), (0.2925544133619187, 0.5885882331079686, 1)]],
 [10, [('tub', 'either'), (1501.2887208655136, 0.0, 1)]],
 [11, [('tub', 'xray'), (748.9914841402323, 6.647975771260783e-165, 1)]],
 [12, [('tub', 'dysp'), (48.41973430768081, 3.4408804621576866e-12, 1)]],
 [13, [('smoke', 'lung'), (389.0571623334361, 1.3278064999089712e-86, 1)]],
 [14, [('smoke', 'bronc'), (882.6671591996876, 5.751917378253763e-194, 1)]],
 [15, [('smoke',